In [1]:
import sys
import os
import tensorflow as tf
import math

In [2]:
# VGG16 es un modelo preentrenado que contiene multitud de imagenes.
# Este modelo preentrado es uno de los muchos que tiene Keras
vgg = tf.keras.applications.vgg16.VGG16()

In [3]:
# Resumen de las capas que tiene la red neuronal VGG16
vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [4]:
# Creación de la red convolucional
cnn = tf.keras.Sequential()

# Se añaden las capas que se ha visto antes a nuestra propia red
for capa in vgg.layers:
    cnn.add(capa)

In [5]:
# Resumen con las capas de nuestra propia red. Debe coincidir con el resumen
# realizado en la red VGG16
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
# Con esto se elimina la capa de predicción. El pop elimina la ultima capa
# del modelo. La ultima capa era la de "predictions" que tiene 1000 parámetros, tipos de imagenes,
# que pued clasificar. Como queremos usarlo para el ejemplo de gatos, gorilas y perros, se quita dicha
# capa para añadir una propia.
cnn.pop()

In [7]:
# Las capas que provienen del VGG16 no queremos entrenarlas porque ya lo han sido
# en el pasado. Por lo tanto, indicamos que no es necesario entrenarlas.
for layer in cnn.layers:
    layer.trainable = False

In [8]:
# Añadimos una capa de predicción de softmax que tendrá 3 neuronas. Ya que se usará el ejemplo
# de gatos, gorilas y perros.
cnn.add(tf.keras.layers.Dense(3,activation='softmax'))

In [9]:
# Esta función encapsula lo que hemos hecho en el código anterior paso a paso.
def modelo():
    vgg=tf.keras.applications.vgg16.VGG16()
    cnn=tf.keras.Sequential()
    for capa in vgg.layers:
        cnn.add(capa)
    cnn.pop()
    for layer in cnn.layers:
        layer.trainable = False
    cnn.add(tf.keras.layers.Dense(3,activation='softmax'))

    return cnn

In [10]:
# Se limpia todas las variables, estado, etc. de sesión que hemos abierto de keras
tf.keras.backend.clear_session()

In [11]:
# Trozo de código del clasificador de imagenes pero aquí no se indicarán las capas convolucionales
data_entrenamiento = "K:/github/python_test/tensorflow/clasificador_imagenes/data/entrenamiento/"
data_validacion = "K:/github/python_test/tensorflow/clasificador_imagenes/data/validacion/"

# Parametros de la red neuronal
# Número de veces que se iteran los datos en el entrenamiento
epocas = 20
# Tamaño de las imagenes. Este es el tamaño que espera la red VGG16, en la documentación
# oficial lo indica.
longitud, altura = 224, 224
# Numero de imagenes que se procesan en cada paso
batch_size = 32
# Al final de cada época se harán 300 pasos con los datos de validación
# para ir viendo como de bien va aprendiendo
validation_steps = 32
# Número de convoluciones o de capas(profundidad) que tendra la iamagen
# Primera convolucion serán 32
filtrosConv1 = 32
# Segunda convolucion serán 64
filtrosConv2 = 64
# Anchura y altura que va a procesar en cada convolucion.
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
# Tamaño que se va usar en el maxpooling
tamano_pool = (2, 2)
# Número de clases que hay en el set de datos: gatos, perros y gorilas
clases = 3
# Es el learning rate. Es decir, cuanto de grandes van a ser los ajustes en la red
# para ajustarse para buscar una solución óptima.
lr = 0.0004

# Como se van a transformar las imagenes para poderlas pasar al procesao de entrenamiento
entrenamiento_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Lo mismo pero para los datos de test
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1. / 255)    

# Proceso de lectura y transformación de los datos para los datos de entrenamiento
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

# Lo mismo para los datos de test
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')    
# Se llama a la función que devolverá el modelo adaptado basandonas en la red BGG16
cnn = modelo()

# Se indica como aprenderá el algoritmo
cnn.compile(loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(lr=lr),
            metrics=['accuracy'])
# Calculo de pasos por epoca
pasos = math.ceil( ( len(entrenamiento_generador.filenames) / batch_size ) )    

cnn.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps )

# Directorio donde se guardará el modelo
target_dir = 'K:/github/python_test/tensorflow/clasificador_imagenes/modelo'
if not os.path.exists(target_dir):
 os.mkdir(target_dir)

# Grabación del modelo
cnn.save('K:/github/python_test/tensorflow/clasificador_imagenes/modelo/modelo.h5')

# Grabación de los pesos de cada una de las capas
cnn.save_weights('K:/github/python_test/tensorflow/clasificador_imagenes/modelo/pesos.h5')    

Found 999 images belonging to 3 classes.
Found 2043 images belonging to 3 classes.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 32 steps, validate for 32 steps
Epoch 1/20
32/32 [==============================] - 264s 8s/step - loss: 0.8149 - accuracy: 0.5756 - val_loss: 0.6778 - val_accuracy: 0.6660
Epoch 2/20
32/32 [==============================] - 296s 9s/step - loss: 0.6485 - accuracy: 0.7247 - val_loss: 0.5450 - val_accuracy: 0.7881
Epoch 3/20
32/32 [==============================] - 323s 10s/step - loss: 0.5639 - accuracy: 0.7778 - val_loss: 0.5101 - val_accuracy: 0.8018
Epoch 4/20
32/32 [==============================] - 317s 10s/step - loss: 0.5222 - accuracy: 0.7938 - val_loss: 0.4942 - val_accuracy: 0.8008
Epoch 5/20
32/32 [==============================] - 315s 10s/step - loss: 0.5067 - accuracy: 0.8088 - val_loss: 0.4539 - val_accuracy: 0.8213
Epoch 6/20
32/32 [==============================] - 300s 9s/step - loss: 0.4845 - accuracy: 0.8078 - val_loss: 0.4486